In [29]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import datetime
import io
from PIL import Image
from IPython.display import Image as IPyImage
import imageio
print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.6.0


In [30]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(1, activation = 'linear', input_dim = 784))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1), loss='mean_squared_error', metrics=['mae'])
    return model

In [31]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()
x_train=x_train.reshape(60000,784).astype('float32')/255
x_test=x_test.reshape(10000,784).astype('float32')/255

In [32]:
class myCallback(tf.keras.callbacks.Callback):
  def on_train_batch_begin(self,batch,logs=None):
    print('training of batch{} begins at {}'.format(batch,datetime.datetime.now().time()))
  def on_train_batch_end(self,batch,logs=None):
    print('training of batch{} ends at {}'.format(batch,datetime.datetime.now().time()))
    

In [33]:
model=get_model()
model.fit(x_train,y_train,epochs=1,steps_per_epoch=5,verbose=0,callbacks=[myCallback()])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


training of batch0 begins at 14:53:55.070617
training of batch0 ends at 14:53:55.319297
training of batch1 begins at 14:53:55.319479
training of batch1 ends at 14:53:55.336283
training of batch2 begins at 14:53:55.336426
training of batch2 ends at 14:53:55.352622
training of batch3 begins at 14:53:55.352759
training of batch3 ends at 14:53:55.364099
training of batch4 begins at 14:53:55.364202
training of batch4 ends at 14:53:55.374635


In [34]:
callback = tf.keras.callbacks.LambdaCallback(
    on_epoch_end=lambda epoch,logs: 
    print("Epoch: {}, Val/Train loss ratio: {:.2f}".format(epoch, logs["val_loss"] / logs["loss"]))
)

model = get_model()
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=64,
          epochs=3,
          verbose=0,
          callbacks=[callback])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch: 0, Val/Train loss ratio: 0.76
Epoch: 1, Val/Train loss ratio: 0.85
Epoch: 2, Val/Train loss ratio: 0.69


In [35]:
class DetectOverfittingCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold=0.7):
        super(DetectOverfittingCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        ratio = logs["val_loss"] / logs["loss"]
        print("Epoch: {}, Val/Train loss ratio: {:.2f}".format(epoch, ratio))

        if ratio > self.threshold:
            print("Stopping training...")
            self.model.stop_training = True

model = get_model()
model.fit(x_train, y_train,
              validation_data=(x_test, y_test),
              batch_size=64,
              epochs=3,
              verbose=0,
              callbacks=[DetectOverfittingCallback()])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch: 0, Val/Train loss ratio: 2.37
Stopping training...


In [36]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

In [52]:
plt.rc('font',size=20)
plt.rc('figure',figsize=(15,3))

def display_digits(inputs,outputs,epoch,ground_truth,n=10):
  plt.clf()

  plt.yticks([])
  plt.grid(None)
  inputs = np.reshape(inputs, [n, 28, 28])
  inputs = np.swapaxes(inputs, 0, 1)
  inputs = np.reshape(inputs, [28, 28*n])
  plt.imshow(inputs)
  plt.xticks([28*x+14 for x in range(n)], outputs)
  for i,t in enumerate(plt.gca().xaxis.get_ticklabels()):
       if outputs[i] == ground_truth[i]: 
           t.set_color('green') 
       else: 
           t.set_color('red')
  plt.grid(None)

In [53]:
GIF_PATH = './animation.gif'

In [54]:
class VisCallback(tf.keras.callbacks.Callback):
    def __init__(self, inputs, ground_truth, display_freq=10, n_samples=10):
        self.inputs = inputs
        self.ground_truth = ground_truth
        self.images = []
        self.display_freq = display_freq
        self.n_samples = n_samples

    def on_epoch_end(self, epoch, logs=None):
        indexes = np.random.choice(len(self.inputs), size=self.n_samples)
        X_test, y_test = self.inputs[indexes], self.ground_truth[indexes]
        predictions = np.argmax(self.model.predict(X_test), axis=1)
        display_digits(X_test, predictions, y_test, epoch, n=self.display_freq)
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image = Image.open(buf)
        self.images.append(np.array(image))
        if epoch % self.display_freq == 0:
            plt.show()

    def on_train_end(self, logs=None):
        imageio.mimsave(GIF_PATH, self.images, fps=1)

In [55]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(32, activation='linear', input_dim=784))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model